In [ ]:
import pandas as pd
from wikiwho_chobj import Chobjer

context = 10
co = Chobjer(article="39570", pickles_path='../../bert', lang='en', context=context)
df_all = pd.DataFrame(co.iter_chobjs(), columns = next(co.iter_chobjs()).keys())

In [ ]:
from nltk.corpus import stopwords 
nltk.download("stopwords")
stop_words = set(stopwords.words('english'))

In [ ]:
import sys
sys.path.insert(1, '../utils/')
from merge import combine

df=df_all[(df_all['ins_tokens_str'].str.len() + df_all['del_tokens_str'].str.len()) <= 20]
jlogie = pd.read_csv("../../John_Logie_Baird_FULL.csv")
merged = df.apply(lambda x: combine(x, jlogie), axis=1)

# captures if we also want to use changeobjects that do not have tokens that are ground-truth labelled
OUTER_JOIN = True

merged = merged.dropna(how="all")
if not OUTER_JOIN:
    merged = merged[(merged["birth_place"].isna() & merged["Bulk"].isna() & merged["nationality"].isna() & merged["Link"].isna())== False]

In [ ]:
def making_list(tokens):
    new_tokens = []
    for item in tokens:
        new_tokens.append(' '.join(word for word in item if not word in stop_words))
                          #.replace('[[', '').replace(']]', '').replace('}}', '').replace('{{', '').replace('–', ''))

    return(new_tokens)


all_tokens = []
for i, row in merged.iterrows():
    all_tokens.append(making_list([row['left_token_str'],row['right_token_str']]))#,row['ins_tokens_str'],row['del_tokens_str']]))


In [ ]:
from bert_serving.client import BertClient
import numpy as np
bc = BertClient()
#c = 0
features = pd.DataFrame()
for row in all_tokens:
    vector=[]
    for token in row:
        if token.isspace() or token == '':
            vector.append(np.full((1, 768), 0))
        else:
            vector.append(bc.encode([token]))

    new_vector = np.concatenate((vector[0], vector[1]#, vector[2], vector[3]
                                ),axis = 1)
    features = features.append(pd.DataFrame(new_vector[0]).T)  
    #c+=1

In [ ]:
from sklearn.cluster import KMeans

clusterer = KMeans(random_state=42)
clusters = clusterer.fit_predict(features)

In [ ]:
pd.Series(clusters).unique()

In [ ]:
from sklearn.manifold import TSNE


X = TSNE(random_state=42).fit_transform(features)

In [ ]:
plot_data = pd.concat([pd.DataFrame(X), pd.Series(clusters), merged], axis=1)

plot_data.columns = ['t-SNE-X', 't-SNE-Y', 'cluster', 'Bulk',
                  'Link',          'action',     'birth_place',
           'del_end_pos',   'del_start_pos',      'del_tokens',
        'del_tokens_str',          'editor',        'from_rev',
        'from_timestamp',     'ins_end_pos',   'ins_start_pos',
            'ins_tokens',  'ins_tokens_str',      'left_neigh',
            'left_token',  'left_token_str',     'nationality',
               'page_id',     'right_neigh',     'right_token',
       'right_token_str',            'text',          'to_rev',
          'to_timestamp',           'token']     

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig = plt.figure(figsize=(16, 10), dpi= 80, facecolor='w', edgecolor='k')
plt.scatter(plot_data["t-SNE-X"], plot_data["t-SNE-Y"], s=10)

## tSNE k-means

In [ ]:
import plotly.io
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import random

init_notebook_mode(connected=True)

r = lambda: random.randint(0,255)

traces = []
for c in plot_data.cluster.unique():
    
   # Create a trace
    trace = go.Scatter(
        x=plot_data[plot_data["cluster"]==c]["t-SNE-X"],
        
        
        y=plot_data[plot_data["cluster"]==c]["t-SNE-Y"],
        
        
        mode = 'markers',
        name = str(c),
        marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r())),
        text = plot_data.index,
        
        showlegend = True,

    )
    traces.append(trace)

data = traces

# Plot and embed in ipython notebook
sys.path.insert(1, '../bert-tfidf/graphs')
iplot(data, filename='kmeans_context_10',image='png', image_height = 500, image_width = 700) 
#plotly.io.write_image("../bert-tfidf/graphs/kmeans-context_", context, ".png")

## Nationality

In [ ]:
# enable javascript support
init_notebook_mode(connected=True)

r = lambda: random.randint(0,255)

traces = []
for c in ["Y", "N", None]:
    
    if c is None:
        trace = go.Scatter(
            x=plot_data.loc[plot_data["nationality"].isna(),"t-SNE-X"],
            y=plot_data.loc[plot_data["nationality"].isna(),"t-SNE-Y"],
            mode = 'markers',
            name = str(c),
            marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r())),
            text = plot_data.index,
            
            showlegend = True,
        )
    else:
        # Create a trace
        trace = go.Scatter(
            x=plot_data.loc[plot_data["nationality"]==c,"t-SNE-X"],
            y=plot_data.loc[plot_data["nationality"]==c,"t-SNE-Y"],
            mode = 'markers',
            name = str(c),
            marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r())),
            text = plot_data.index,
            showlegend = True,
        )
    traces.append(trace)

data = traces

# Plot and embed in ipython notebook
iplot(data, filename='basic-scatter')

## tSNE DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
db = DBSCAN(eps=9.5, min_samples=10).fit(features)
labels = db.labels_

# number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

In [ ]:
X = TSNE(random_state=42).fit_transform(features)

In [ ]:
plot_data_dbscan = pd.concat([pd.DataFrame(X), pd.Series(labels), merged], axis=1)

plot_data_dbscan.columns = ['t-SNE-X', 't-SNE-Y', 'db_cluster', 'Bulk',
                  'Link',          'action',     'birth_place',
           'del_end_pos',   'del_start_pos',      'del_tokens',
        'del_tokens_str',          'editor',        'from_rev',
        'from_timestamp',     'ins_end_pos',   'ins_start_pos',
            'ins_tokens',  'ins_tokens_str',      'left_neigh',
            'left_token',  'left_token_str',     'nationality',
               'page_id',     'right_neigh',     'right_token',
       'right_token_str',            'text',          'to_rev',
          'to_timestamp',           'token']   

In [ ]:


r = lambda: random.randint(0,255)

traces = []
for c in plot_data_dbscan.db_cluster.unique():
    if c== -1:
        trace = go.Scatter(
            x=plot_data_dbscan[plot_data_dbscan["db_cluster"]==c]["t-SNE-X"],
            y=plot_data_dbscan[plot_data_dbscan["db_cluster"]==c]["t-SNE-Y"],
            mode = 'markers',
            name = str(c),
            marker = go.scatter.Marker(size=4, color='grey'),
            text = plot_data_dbscan.index,
            showlegend = True,
        )
    else:
   # Create a trace
        trace = go.Scatter(
            x=plot_data_dbscan[plot_data_dbscan["db_cluster"]==c]["t-SNE-X"],
            y=plot_data_dbscan[plot_data_dbscan["db_cluster"]==c]["t-SNE-Y"],
            mode = 'markers',
            name = str(c),
            marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r())),
            text = plot_data_dbscan.index,
            showlegend = True,
        )
    traces.append(trace)

data = traces

# Plot and embed in ipython notebook
sys.path.insert(1, '../bert-tfidf/plots/')
iplot(data, filename='dbscan_context_10',image='png', image_height = 500, image_width = 700) 

## Lasso selection with printing elements in a qgrid table

In [ ]:
DISPLAYED_TABLE_COLUMNS = ["t-SNE-X", "t-SNE-Y", "left_token_str", "right_token_str", "del_tokens_str", "ins_tokens_str"]

In [ ]:
import time
import qgrid
qgrid.set_grid_option('maxVisibleRows', 5)

import pdb

f = go.FigureWidget()
f.layout.dragmode = 'lasso'

scatter = f.add_scatter(x = plot_data["t-SNE-X"], y = plot_data["t-SNE-Y"], mode = 'markers', 
                       marker=go.scatter.Marker(size=0, opacity=0),showlegend = False)


r = lambda: random.randint(0,255)

# Create a table FigureWidget that updates on selection from points in the scatter plot of f
t = go.FigureWidget([go.Table(  
    header=dict(values=DISPLAYED_TABLE_COLUMNS,             
                
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    
    cells=dict(values=[plot_data[col] for col in DISPLAYED_TABLE_COLUMNS],              
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5
               ))])

def selection_fn(trace,points,selector):   
    with out:
        clear_output()
        #display(plot_data.loc[points.point_inds])
        
        display(qgrid.show_grid(plot_data.loc[points.point_inds, DISPLAYED_TABLE_COLUMNS]))
    
    t.data[0].cells.values = [plot_data.loc[points.point_inds][col] for col in DISPLAYED_TABLE_COLUMNS]

traces = []
for c in plot_data["cluster"].unique():
    # Create a trace
    trace = go.Scatter(
        x=plot_data[plot_data["cluster"] == c]["t-SNE-X"],
        y=plot_data[plot_data["cluster"] == c]["t-SNE-Y"],
        mode = 'markers',
        name = str(c),
        uid = str(c),
        marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r())),
        text = plot_data.index,
        
        showlegend = True, 

    )
    trace.on_selection(selection_fn)
    
    f.add_trace(trace)

    
scatter.on_selection(selection_fn)

from IPython.display import display, clear_output
display(f)

from ipywidgets import widgets, Output
out = Output()
display(out)